after that need to restart the runtime

In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.0 MB/s 
     |████████████████████████████████| 79 kB 5.9 MB/s 
     |████████████████████████████████| 88 kB 8.0 MB/s 
     |████████████████████████████████| 5.2 MB 44.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.2 MB 40.3 MB/s 
     |████████████████████████████████| 1.2 MB 50.2 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 48.0 MB/s 
     |████████████████████████████████| 636 kB 56.5 MB/s 
     |████████████████████████████████| 1.1 MB 36.8 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2342163 sha256=6e532ece2fa27ea49171b171d79a14c88e82bbf0f561c4704b175df6ea9368d4
  Stored in directory: /root/.cache/p

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 15.4 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 788 kB 57.3 MB/s 
     |████████████████████████████████| 19.7 MB 1.3 MB/s 
     |████████████████████████████████| 1.2 MB 38.0 MB/s 
     |████████████████████████████████| 981 kB 21.8 MB/s 
     |████████████████████████████████| 46 kB 2.9 MB/s 
     |████████████████████████████████| 1.6 MB 13.8 MB/s 
     |████████████████████████████████| 199 kB 45.4 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=263b877229f2fb3508ddc801d260efc48eb424a1f59378b35ba2b5186e02caf6
  Stored in directory: /root/.cache/pip/wheels/26/70/6a/1c79e59951a41b4045497da187b2724f5659ca64033cf4548e
  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl 

need to connect to the drive link : https://drive.google.com/drive/folders/1bs8hVgf56qeI028D0TmaYQK01-BTT4Hr?usp=sharing and add it as folder so we can get to the data of the tweets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


import

In [ ]:
import re
from bertopic import BERTopic
import datetime as dt
import json
from flair.embeddings import TransformerDocumentEmbeddings
from umap import UMAP
from hdbscan import HDBSCAN
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


class to parser we will improve it.

In [ ]:
stop_word_heb = ["אבל", "או", "אולי", "אותה", "אותו", "אותי", "אותך", "אותם", "אותן", "אותנו", "אז", "אחר", "אחרות",
                 "אחרי", "אחריכן", "אחרים", "אחרת", "אי", "איזה", "איך", "אין", "איפה", "איתה", "איתו", "איתי", "איתך",
                 "איתכם", "איתכן", "איתם", "איתן", "איתנו", "אך", "אל", "אלה", "אלו", "אם", "אנחנו", "אני", "אס", "אף",
                 "אצל", "אשר", "את", "אתה", "אתכם", "אתכן", "אתם", "אתן", "באיזו מידה", "באמצע", "באמצעות", "בגלל",
                 "בין", "בלי", "במידה", "במקום שבו", "ברם", "בשביל", "בשעהש", "בתוך", "גם", "דרך", "הוא", "היא", "היה",
                 "היכן", "היתה", "היתי", "הם", "הן", "הנה", "הסיבה שבגללה", "הרי", "ואילו", "ואת", "זאת", "זה", "זות",
                 "יהיה", "יוכל", "יוכלו", "יותר מדי", "יכול", "יכולה", "יכולות", "יכולים", "יכל", "יכלה", "יכלו", "יש",
                 "כאן", "כאשר", "כולם", "כולן", "כזה", "כי", "כיצד", "כך", "ככה", "כל", "כלל", "כמו", "כן", "כפי", "כש",
                 "לא", "לאו", "לאיזו תכלית", "לאן", "לבין", "לה", "להיות", "להם", "להן", "לו", "לי", "לכם", "לכן",
                 "למה", "למטה", "למעלה", "למקום שבו", "למרות", "לנו", "לעבר", "לעיכן", "לפיכך", "לפני", "מאד", "מאחורי",
                 "מאיזו סיבה", "מאין", "מאיפה", "מבלי", "מבעד", "מדוע", "מה", "מהיכן", "מול", "מחוץ", "מי", "מכאן",
                 "מכיוון", "מלבד", "מן", "מנין", "מסוגל", "מעט", "מעטים", "מעל", "מצד", "מקום בו", "מתחת", "מתי", "נגד",
                 "נגר", "נו", "עד", "עז", "על", "עלי", "עליה", "עליהם", "עליהן", "עליו", "עליך", "עליכם", "עלינו", "עם",
                 "עצמה", "עצמהם", "עצמהן", "עצמו", "עצמי", "עצמם", "עצמן", "עצמנו", "פה", "רק", "שוב", "של", "שלה",
                 "שלהם", "שלהן", "שלו", "שלי", "שלך", "שלכה", "שלכם", "שלכן", "שלנו", "שם", "תהיה", "תחת"]
stop_word_heb2=[]
for w in stop_word_heb:
  stop_word_heb2.append(" "+w+" ")

print(stop_word_heb2)

[' אבל ', ' או ', ' אולי ', ' אותה ', ' אותו ', ' אותי ', ' אותך ', ' אותם ', ' אותן ', ' אותנו ', ' אז ', ' אחר ', ' אחרות ', ' אחרי ', ' אחריכן ', ' אחרים ', ' אחרת ', ' אי ', ' איזה ', ' איך ', ' אין ', ' איפה ', ' איתה ', ' איתו ', ' איתי ', ' איתך ', ' איתכם ', ' איתכן ', ' איתם ', ' איתן ', ' איתנו ', ' אך ', ' אל ', ' אלה ', ' אלו ', ' אם ', ' אנחנו ', ' אני ', ' אס ', ' אף ', ' אצל ', ' אשר ', ' את ', ' אתה ', ' אתכם ', ' אתכן ', ' אתם ', ' אתן ', ' באיזו מידה ', ' באמצע ', ' באמצעות ', ' בגלל ', ' בין ', ' בלי ', ' במידה ', ' במקום שבו ', ' ברם ', ' בשביל ', ' בשעהש ', ' בתוך ', ' גם ', ' דרך ', ' הוא ', ' היא ', ' היה ', ' היכן ', ' היתה ', ' היתי ', ' הם ', ' הן ', ' הנה ', ' הסיבה שבגללה ', ' הרי ', ' ואילו ', ' ואת ', ' זאת ', ' זה ', ' זות ', ' יהיה ', ' יוכל ', ' יוכלו ', ' יותר מדי ', ' יכול ', ' יכולה ', ' יכולות ', ' יכולים ', ' יכל ', ' יכלה ', ' יכלו ', ' יש ', ' כאן ', ' כאשר ', ' כולם ', ' כולן ', ' כזה ', ' כי ', ' כיצד ', ' כך ', ' ככה ', ' כל ', ' כלל ', ' כמו 

In [ ]:
class datapre:
    re_to_re=['\'','\"','_',"-"]
    re_to_remove = ['http\S+', '\n', '#\S+','\r','[A-Z]+','[a-z]+']#, '#\S+'
    stop_word_heb = [' אבל ', ' או ', ' אולי ', ' אותה ', ' אותו ', ' אותי ', ' אותך ', ' אותם ', ' אותן ', ' אותנו ', ' אז ', ' אחר ', ' אחרות ', ' אחרי ', ' אחריכן ', ' אחרים ', ' אחרת ', ' אי ', ' איזה ', ' איך ', ' אין ', ' איפה ', ' איתה ', ' איתו ', ' איתי ', ' איתך ', ' איתכם ', ' איתכן ', ' איתם ', ' איתן ', ' איתנו ', ' אך ', ' אל ', ' אלה ', ' אלו ', ' אם ', ' אנחנו ', ' אני ', ' אס ', ' אף ', ' אצל ', ' אשר ', ' את ', ' אתה ', ' אתכם ', ' אתכן ', ' אתם ', ' אתן ', ' באיזו מידה ', ' באמצע ', ' באמצעות ', ' בגלל ', ' בין ', ' בלי ', ' במידה ', ' במקום שבו ', ' ברם ', ' בשביל ', ' בשעהש ', ' בתוך ', ' גם ', ' דרך ', ' הוא ', ' היא ', ' היה ', ' היכן ', ' היתה ', ' היתי ', ' הם ', ' הן ', ' הנה ', ' הסיבה שבגללה ', ' הרי ', ' ואילו ', ' ואת ', ' זאת ', ' זה ', ' זות ', ' יהיה ', ' יוכל ', ' יוכלו ', ' יותר מדי ', ' יכול ', ' יכולה ', ' יכולות ', ' יכולים ', ' יכל ', ' יכלה ', ' יכלו ', ' יש ', ' כאן ', ' כאשר ', ' כולם ', ' כולן ', ' כזה ', ' כי ', ' כיצד ', ' כך ', ' ככה ', ' כל ', ' כלל ', ' כמו ', ' כן ', ' כפי ', ' כש ', ' לא ', ' לאו ', ' לאיזו תכלית ', ' לאן ', ' לבין ', ' לה ', ' להיות ', ' להם ', ' להן ', ' לו ', ' לי ', ' לכם ', ' לכן ', ' למה ', ' למטה ', ' למעלה ', ' למקום שבו ', ' למרות ', ' לנו ', ' לעבר ', ' לעיכן ', ' לפיכך ', ' לפני ', ' מאד ', ' מאחורי ', ' מאיזו סיבה ', ' מאין ', ' מאיפה ', ' מבלי ', ' מבעד ', ' מדוע ', ' מה ', ' מהיכן ', ' מול ', ' מחוץ ', ' מי ', ' מכאן ', ' מכיוון ', ' מלבד ', ' מן ', ' מנין ', ' מסוגל ', ' מעט ', ' מעטים ', ' מעל ', ' מצד ', ' מקום בו ', ' מתחת ', ' מתי ', ' נגד ', ' נגר ', ' נו ', ' עד ', ' עז ', ' על ', ' עלי ', ' עליה ', ' עליהם ', ' עליהן ', ' עליו ', ' עליך ', ' עליכם ', ' עלינו ', ' עם ', ' עצמה ', ' עצמהם ', ' עצמהן ', ' עצמו ', ' עצמי ', ' עצמם ', ' עצמן ', ' עצמנו ', ' פה ', ' רק ', ' שוב ', ' של ', ' שלה ', ' שלהם ', ' שלהן ', ' שלו ', ' שלי ', ' שלך ', ' שלכה ', ' שלכם ', ' שלכן ', ' שלנו ', ' שם ', ' תהיה ', ' תחת ']

    # stop_word_heb=[]

    re_to_remove_all=re_to_remove+stop_word_heb
    def clean_text(self,text):
        cleantext = re.sub('|'.join(self.re_to_remove_all), ' ', text)
        cleantext = re.sub('|'.join(self.re_to_re), '', cleantext)
        cleantext = re.sub(r'[^\w]', ' ', cleantext)
        cleantext = re.sub(' +', ' ', cleantext)
        return cleantext

class to get the data

In [ ]:
class datalist:
    text_raw=[]
    text_train=[]
    text_time=[]
    tweet_id=[]
    uniqe={}
    pre=datapre()
    def __init__(self):
        self.text_train=[]
        self.text_time=[]
        self.tweet_id=[]
        self.uniqe={}
        self.pre=datapre()
    def put_in_cluster(self,fileline):
        for line in fileline:
            t = json.loads(line)
            text = None
            if 'lang' in t:# משום מה יש ציוצים שאין להם בכלל שדא של שפה
              if t['lang'] == 'iw' or t['lang'] == 'he':#בודק ששפה היא עברית
                  if 'retweeted_status' in t:#check if it retweet
                    tweett=t['retweeted_status']#משנה את המידע לעבוד על הציוץ המקורי
                  else:
                    tweett=t#זה הציוץ המקורי במקרה הזה

                  if tweett['id'] not in self.uniqe:#בודק שזה פעם ראשונה שציוץ מופיע
                    self.uniqe[tweett['id']]=1
                    if 'extended_tweet' in tweett:#במקרה שיש טקסט מלא לוקח את הטקסט המלא זה קורה במקרים שטקסט ארוך מדי
                        text = tweett['extended_tweet']['full_text']
                    else:
                        text = tweett['text']

                    cleantext = self.pre.clean_text(text)
                    if len(set(cleantext.split()))<=6:
                      continue
                    self.text_train.append(cleantext)
                    self.text_raw.append(re.sub('http\S+', '', text))
                    self.tweet_id.append(t['id'])
                    new_datetime = dt.datetime.strftime(dt.datetime.strptime(t["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
                    self.text_time.append(new_datetime)
                  else:
                    self.uniqe[tweett['id']]+=1



    def get_data_cluster(self,files_path, start_date='2019-01-15', end_date='2019-02-01'):
        delta = dt.timedelta(days=1)
        time = dt.datetime.strptime(start_date, "%Y-%m-%d")
        endtime = dt.datetime.strptime(end_date, "%Y-%m-%d")
        self.text_train=[]
        while time <= endtime:
            file_name = files_path + "/Hebrew_tweets.json." + time.strftime("%Y-%m-%d") + ".txt"
            file = open(file_name, "r")
            fileline = file.readlines()
            self.put_in_cluster(fileline)
            print(time)
            time += delta


class for the model

In [ ]:
class topic_model:
    # embedding = TransformerDocumentEmbeddings('onlplab/alephbert-base'),nr_topics="auto"
    umap_model = UMAP(n_neighbors=15, n_components=10, min_dist=0.0, metric='cosine')
    hdbscan_model = HDBSCAN(min_cluster_size=40,min_samples=20, metric='manhattan', cluster_selection_method='leaf', prediction_data=True)
    topic_model = BERTopic(language="multilingual",top_n_words=20,umap_model=umap_model,hdbscan_model=hdbscan_model, calculate_probabilities=True, verbose=True)#, embedding_model="paraphrase-multilingual-MiniLM-L12-v2"

    def __init__(self,name=""):
        self.name = name
        self.datalst= datalist()
        self.topics, self.probs=None,None
    def get_data(self,path, start_date='2019-02-01', end_date='2019-02-07'):
        self.datalst.get_data_cluster(path,start_date,end_date)
    def train(self):
        self.topics, self.probs= self.topic_model.fit_transform(self.datalst.text_raw)
    def upadteTopicByNgrams(self,minNgrams,maxNgrams):
        # vectorizer_model = CountVectorizer(ngram_range=(minNgrams, maxNgrams), stop_words=self.stop_word_heb)
        # self.topic_model = BERTopic(vectorizer_model=vectorizer_model).fit(self.datalst.text_train)
        self.topic_model.update_topics(self.datalst.text_train, self.topics, n_gram_range=(minNgrams, maxNgrams))
    def upadteTopicdata(self,data,minNgrams,maxNgrams):
        self.topic_model.update_topics(data, self.topics, n_gram_range=(minNgrams, maxNgrams))
    def save(self,path="/content/drive/MyDrive/פרויקט סיימנר פרויקט/מאגר המידע/berttopic"):
        self.topic_model.save(path+"\\my_model"+self.name)
        with open(path+'/docs'+self.name+'.pickle', 'wb') as handle:
          pickle.dump(self.datalst.text_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(path+'/docsraw'+self.name+'.pickle', 'wb') as handle:
          pickle.dump(self.datalst.text_raw, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(path+'/docstime'+self.name+'.pickle', 'wb') as handle:
          pickle.dump(self.datalst.text_time, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(path+'/docsid'+self.name+'.pickle', 'wb') as handle:
          pickle.dump(self.datalst.tweet_id, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(path+'/uniqe'+self.name+'.pickle', 'wb') as handle:
          pickle.dump(self.datalst.uniqe, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(path+'/topic'+self.name+'.pickle', 'wb') as handle:
          pickle.dump(self.topics, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(path+'/prob'+self.name+'.pickle', 'wb') as handle:
          pickle.dump(self.probs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    def load(self,path="/content/drive/MyDrive/פרויקט סיימנר פרויקט/מאגר המידע/berttopic"):
        self.topic_model = BERTopic.load(path+"\\my_model"+self.name)
        with open(path+'/docs'+self.name+'.pickle', "rb") as input_file:
          self.datalst.text_train=pickle.load(input_file)
        with open(path+'/docsraw'+self.name+'.pickle', "rb") as input_file:
          self.datalst.text_raw=pickle.load(input_file)
        with open(path+'/docstime'+self.name+'.pickle', 'rb') as input_file:
          self.datalst.text_time=pickle.load(input_file)
        with open(path+'/docsid'+self.name+'.pickle', 'rb') as input_file:
          self.datalst.tweet_id=pickle.load(input_file)
        with open(path+'/uniqe'+self.name+'.pickle', 'rb') as input_file:
          self.datalst.uniqe=pickle.load(input_file)
        with open(path+'/topic'+self.name+'.pickle', "rb") as input_file:
          self.topics=pickle.load(input_file)
        with open(path+'/prob'+self.name+'.pickle', "rb") as input_file:
          self.probs=pickle.load(input_file)

    def get_len(self):
      return (len(self.datalst.text_train))

train the model need to put the drive loction of the tweet files

In [ ]:
model= topic_model("2")
model.get_data('/content/drive/MyDrive/פרויקט סיימנר פרויקט/מאגר המידע/output_folder',start_date='2019-02-01', end_date='2019-02-07')
model.train()

2019-02-01 00:00:00
2019-02-02 00:00:00
2019-02-03 00:00:00
2019-02-04 00:00:00
2019-02-05 00:00:00
2019-02-06 00:00:00
2019-02-07 00:00:00


Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Batches:   0%|          | 0/95 [00:00<?, ?it/s]

2022-05-29 07:23:58,335 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-05-29 07:24:24,167 - BERTopic - Reduced dimensionality
2022-05-29 07:24:24,491 - BERTopic - Clustered reduced embeddings


get topic by ngrams.

In [ ]:
model.upadteTopicByNgrams(2,3)
topic_modell=model.topic_model

the 20 topic with the most document in them. -1 refers to all outliers and should typically be ignored.

In [ ]:
len(model.datalst.text_train)

3015

In [ ]:

df = topic_modell.get_topic_freq()
df = df.drop(labels=[0,1], axis=0)
freq_dict = df.to_dict()
freq_dict = freq_dict['Count']
freq_dict = dict(zip(df.Topic, df.Count))
print(freq_dict)
mean = df["Count"].mean()
max = df["Count"].max()
avg_max = (mean+max)/2
print(str(avg_max))
print("avg: "+ str(df["Count"].mean()))
print("max: "+ str(df["Count"].nlargest(2)))


{1: 191, 2: 138, 3: 125, 4: 105, 5: 104, 6: 93, 7: 81, 8: 72, 9: 58, 10: 51, 11: 48}
143.95454545454544
avg: 96.9090909090909
max: 2    191
3    138
Name: Count, dtype: int64


In [ ]:
freq = topic_modell.get_topic_info(); freq.head(20)

,Topic,Count,Name
0,-1,1668,-1_מדינת ישראל_ראש הממשלה_הגיע הזמן_מפלגת העבודה
1,0,281,0_ראש ממשלה_מפלגת העבודה_ראש הממשלה_אחוז החסימה
2,1,191,1_פיגמת פסים_תכין פיגמת_תכין פיגמת פסים_בלה בלה
3,2,138,2_מפלגת זהות_במדינת ישראל_בני גנץ_ראש הממשלה
4,3,125,3_הבית היהודי_עוצמה יהודית_הרב רפי פרץ_הרב רפי
5,4,105,4_כתב אישום_ראש הממשלה_ראש הממשלה נתניהו_הממשל...
6,5,104,5_מוזמנים לצפות_טוב מייד_האזינו לראיון_טוב מיי...
7,6,93,6_תודה רבה_מזל טוב_התמיכה האדירה_ברוך הבא
8,7,81,7_מדינת ישראל_שבת שלום_ארץ ישראל_תנו את
9,8,72,8_יאיר לפיד_ביומו עבודת_השרים לחקיקה_קהילה תוס...


get the topic top ngrams  the number us the cluster number

In [ ]:
topic_modell.get_topic(1)  # Select the most frequent topic

[('פיגמת פסים', 0.011874512285549199),
 ('תכין פיגמת', 0.009568573246404498),
 ('תכין פיגמת פסים', 0.009568573246404498),
 ('בלה בלה', 0.005366694425294101),
 ('מזיין המוח', 0.004581907296803635),
 ('זיין מוח', 0.004581907296803635),
 ('כיפה אדומה', 0.004418542006579159),
 ('ביבי ביבי', 0.0041883266716769285),
 ('זו הסיבה', 0.004100817105601928),
 ('שר אוצר', 0.003958170761849733),
 ('ענת ברקו', 0.003898377611970536),
 ('אחוז החסימה', 0.0033565264402148484),
 ('מדחת יוסוף', 0.0032081213919296562),
 ('לקלאבוש קרוק', 0.0032081213919296562),
 ('אגם הדרעק', 0.0032081213919296562),
 ('מה רע', 0.0032081213919296562),
 ('וזדיין לקלאבוש', 0.0032081213919296562),
 ('זו הסיבה שלא', 0.0032081213919296562),
 ('יותר מהר', 0.0032081213919296562),
 ('לליכוד כנראה', 0.0032081213919296562)]

In [ ]:
topic_modell.visualize_topics()

In [ ]:
# topic_modell.visualize_distribution(model.probs[155], min_probability=0.005)

In [ ]:
topic_modell.visualize_hierarchy(top_n_topics=50)

In [ ]:
print(len(model.topics))
print(len(model.datalst.text_train))
print(len(model.datalst.text_time))

3015
3015
3015


In [ ]:
topic_modell.visualize_barchart(top_n_topics=10,n_words=20)

In [ ]:
topics_over_time = topic_modell.topics_over_time(docs=model.datalst.text_train,
                                                topics=model.topics,
                                                timestamps=model.datalst.text_time,
                                                global_tuning=True,
                                                evolution_tuning=True,
                                                nr_bins=20)


20it [00:08,  2.47it/s]


In [ ]:
topic_modell.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [ ]:
df = topic_modell.get_topic_freq()
df = df.drop(labels=[0,1], axis=0)
freq_dict = df.to_dict()
freq_dict = freq_dict['Count']
freq_dict = dict(zip(df.Topic, df.Count))
print(freq_dict)
mean = df["Count"].mean()
max = df["Count"].max()
avg_max = (mean+max)/2
print(str(avg_max))
print("avg: "+ str(df["Count"].mean()))
print("max: "+ str(df["Count"].nlargest(2)))

{1: 191, 2: 138, 3: 125, 4: 105, 5: 104, 6: 93, 7: 81, 8: 72, 9: 58, 10: 51, 11: 48}
143.95454545454544
avg: 96.9090909090909
max: 2    191
3    138
Name: Count, dtype: int64


In [ ]:
with open('docs.pickle', 'wb') as handle:
    pickle.dump(model.datalst.text_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('topic.pickle', 'wb') as handle:
    pickle.dump(model.topics, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('prob.pickle', 'wb') as handle:
    pickle.dump(model.probs, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('topicinfo.pickle', 'wb') as handle:
    pickle.dump(freq, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
freq

,Topic,Count,Name
0,-1,1668,-1_מדינת ישראל_ראש הממשלה_הגיע הזמן_מפלגת העבודה
1,0,281,0_ראש ממשלה_מפלגת העבודה_ראש הממשלה_אחוז החסימה
2,1,191,1_פיגמת פסים_תכין פיגמת_תכין פיגמת פסים_בלה בלה
3,2,138,2_מפלגת זהות_במדינת ישראל_בני גנץ_ראש הממשלה
4,3,125,3_הבית היהודי_עוצמה יהודית_הרב רפי פרץ_הרב רפי
5,4,105,4_כתב אישום_ראש הממשלה_ראש הממשלה נתניהו_הממשל...
6,5,104,5_מוזמנים לצפות_טוב מייד_האזינו לראיון_טוב מיי...
7,6,93,6_תודה רבה_מזל טוב_התמיכה האדירה_ברוך הבא
8,7,81,7_מדינת ישראל_שבת שלום_ארץ ישראל_תנו את
9,8,72,8_יאיר לפיד_ביומו עבודת_השרים לחקיקה_קהילה תוס...


In [ ]:
model.save()

In [ ]:
model= topic_model("1")

model.load()

In [ ]:
# model.get_data('/content/drive/MyDrive/פרויקט סיימנר פרויקט/מאגר המידע/output_folder','2019-02-02','2019-02-12')
# model.train()

In [ ]:
freq = model.topic_model.get_topic_info(); freq.head(20)

,Topic,Count,Name
0,-1,1450,-1_הבית היהודי_מדינת ישראל_בני גנץ_הגיע הזמן
1,0,114,0_כתב אישום_ראש הממשלה_ראש הממשלה נתניהו_הממשל...
2,1,92,1_מוזמנים לצפות_פייק ניוז_ערוץ 20_מייד אתראיין
3,2,65,2_וחברי הליכוד_צאו להצביע_לא פי_חברות וחברי הל...
4,3,64,3_ימין ולא שמאל_ולא שמאל_ימין ולא_שומרי הבית
5,4,61,4_120 אלף_אלף קולות_חופש הצבעה_קולות מתוך
6,5,60,5_מדינת ישראל_שבת שלום_את הכח_תודה תודה
7,6,59,6_ראש הממשלה_ראש ממשלה_מבין הסורגים ממש_האישום...
8,7,57,7_עוצמה יהודית_לעוצמה יהודית_קרע בעם_הציונות ה...
9,8,55,8_סוף סוף_סוף סוף נמצא_המדינה עוברת_שנלחם ורץ


In [ ]:
model.topic_model.get_topic(5)

[('מדינת ישראל', 0.015361444723378033),
 ('שבת שלום', 0.013529644298716104),
 ('את הכח', 0.00775565448014746),
 ('תודה תודה', 0.007436937997996306),
 ('תנו את', 0.007436937997996306),
 ('ראש הממשלה', 0.007220582659524424),
 ('למען אזרחי', 0.0071897983300814525),
 ('ארץ ישראל', 0.00622122565085208),
 ('אזרחי ישראל', 0.005603622358442094),
 ('ישראל להישגים', 0.005469999329368181),
 ('תודה למאות הפעילים', 0.005469999329368181),
 ('תנו את הכח', 0.005469999329368181),
 ('לחזק הליכוד', 0.005469999329368181),
 ('בית ישראל', 0.005469999329368181),
 ('שיצאו הבוקר', 0.005469999329368181),
 ('ישראל תודה', 0.005469999329368181),
 ('שבת שלום ומבורך', 0.005469999329368181),
 ('למאות הפעילים שיצאו', 0.005469999329368181),
 ('הבוקר לחבר הישראלים', 0.005469999329368181),
 ('שלום תודה למאות', 0.005469999329368181)]

In [ ]:
# datatt=datalist()
# datatt.get_data_cluster('/content/drive/MyDrive/פרויקט סיימנר פרויקט/מאגר המידע/output_folder','2019-02-02','2019-02-12')

In [ ]:
# model.upadteTopicdata(datatt.text_train,2,3)

In [ ]:
model.topic_model.visualize_barchart(top_n_topics=55,n_words=20,height=750,width=500)

In [ ]:
a=model.datalst.text_raw

b=model.topics
c=model.datalst.tweet_id
percentile_list = pd.DataFrame(
    {'topic': b,
     'text': a,
     "id": c,
    })

In [ ]:
percentile_list.loc[percentile_list['topic'] == 13]

,topic,text,id
16,13,תראו מה קורה כשאודי סגל מרגיז את איילת שקד.\nא...,1091106597764505600
88,13,@gershuni @ZehutLiberals מקווה שזה לא מוציא לך...,1091216573141004288
236,13,אין יותר קפה הוא תה זה הולך ביחד....נו באמת,1091276817347035138
262,13,הלך עליך בא גנץ עושה לך כיפה אדומה אתם לפני פי...,1091295887840735232
408,13,"@Soniac55 קוטג׳ 1,גבינה לבנה 1, כוס גבנ״צ או ק...",1091375355762536448
619,13,את לא שווה שערה מהתחת של דפנה ליף.\nניצלת את ה...,1091742470969397248
683,13,"סוף להכפשות נגד דיכטר\nעל הפקרת מדחת יוסוף ע""י...",1091769084298125314
734,13,מומלץ לראות את זה בלי סאונד...וכל אחד חופשי לפ...,1091790120498446338
956,13,תמשיכו ללקק לביבי והוא יישתה לכם את המנדטים,1092028112739725312
1000,13,שוב ביבי מרים ל @gantzbe \nלהנחתה.\n\nקח תאכל ...,1092051977566076928


In [ ]:
model2= topic_model("2")

model2.load()

In [ ]:
model2.topic_model.visualize_barchart(top_n_topics=38,n_words=20,height=750,width=500)

In [ ]:
a2=model2.datalst.text_raw

b2=model2.topics
c2=model2.datalst.tweet_id
percentile_list = pd.DataFrame(
    {'topic': b2,
     'text': a2,
     "id": c2,
    })

In [ ]:
percentile_list.loc[percentile_list['topic'] == 0]

,topic,text,id
2,0,"איך זה יהיה חיבור של ה""שמאל ציוני"" אם יש שם מ...",1091093823231533056
20,0,מי שרוצה להצביע ימין וגם לקבל ימין צריך להצביע...,1091109959931023360
28,0,"משמח לשמוע כתבים שלמרות שלא מסכימים איתי, מכבד...",1091118674302431232
37,0,תנו לאיש הזה להיות שר החינוך. יש לו יופי של ער...,1091126521627140096
40,0,"כאילו שזה תלוי בה, הימין החדש מפלגת לווין שכל ...",1091129934628749314
...,...,...,...
2986,0,מצביעים כדי להחליף שלטון חבל לבזבז קול מיותר,1093608180771422208
2993,0,@meravi11 @MichalRosenfel1 @HassonYoel מיכלי י...,1093610654014078976
2996,0,ועכשיו מצטרפת מהפכנית נוספת 😉 \nאלונה ברקת לקח...,1093610904753704960
3000,0,אין ספק שאיתן כבל מוכן לעשו. כל תרגיל ותכסיס כ...,1093614552107880448
